In [1]:
import os
from os.path import abspath
import findspark 
findspark.init() 
from pyspark.sql import SparkSession

os.environ['PYSPARK_SUBMIT_ARGS']='--driver-memory 8G --executor-memory 8G pyspark-shell'
spark=SparkSession.builder.appName("ETL pipeline")\
.config("spark.sql.warehouse.dir", abspath('data/spark-warehouse'))\
.config('spark.driver.extraJavaOptions',f'-Dderby.system.home={abspath("data/catalog")}')\
.config("spark.io.encryption.enabled",True)\
.enableHiveSupport().getOrCreate()

sc=spark.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/01/08 13:01:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/01/08 13:01:38 WARN SparkEnv: I/O encryption enabled without RPC encryption: keys will be visible on the wire.


In [3]:
df=spark.read.parquet('data/order')
df.createOrReplaceTempView("order")



In [64]:
### testing 1 endpoint ###

# spark.sql('select distinct delivery_address_city from order where delivery_address_state="AL"').show()
# spark.sql("""select  delivery_address_state,delivery_address_city, count(*) as num_orders
# from order group by delivery_address_state,delivery_address_city
# order by delivery_address_state,delivery_address_city""").show(1000,False)


### code 1 endpoint ###

spark.sql("""select  delivery_address_state,delivery_address_city, date(order_created_at), count(*) as num_orders
from order group by delivery_address_state,delivery_address_city,date(order_created_at) 
order by delivery_address_state,delivery_address_city,date(order_created_at)""").show(1000,False)

In [124]:
### testing 2 endpoint ###

### code 2 endpoint ###

spark.sql("""
with customer_orders as (
    select c.customer_id,c.customer_name,o.merchant_id
    from consumer c
    left join order o 
    on c.customer_id = o.customer_id
    ),
customer_total_restaurants as (
    select co.customer_id,co.customer_name, r.id restaurant_id, count(*) total_orders
    from customer_orders co
    left join  restaurant r
    on co.merchant_id=r.id
    group by co.customer_id,co.customer_name, r.id
    ),
restaurant_rank as (
    select *, 
    row_number() over (partition by customer_id  order by total_orders desc) rank 
    from customer_total_restaurants
)

select * 
from restaurant_rank
where rank<=10

""" ).show()



In [120]:
spark.catalog.listTables()

[Table(name='consumer', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='restaurant', database='default', description=None, tableType='MANAGED', isTemporary=False),
 Table(name='customer_total_restaurants', database=None, description=None, tableType='TEMPORARY', isTemporary=True),
 Table(name='order', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]